# 1. Import and Install Dependencies

In [ ]:
# !pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [ ]:
# !pip3 install mediapipe

# SLT Class 

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [35]:
class SLT:
    
    def __init__(self):
        self.mp_holistic = mp.solutions.holistic # Holistic model
        self.mp_drawing = mp.solutions.drawing_utils # Drawing utilities
        
    
    def mediapipe_detection(self, image, model):
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
        image.flags.writeable = False                  # Image is no longer writeable
        results = model.process(image)                 # Make prediction
        image.flags.writeable = True                   # Image is now writeable 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
        return image, results
    
    
    
    def draw_styled_landmarks(self, image, results):
        # Draw face connections
        self.mp_drawing.draw_landmarks(image, results.face_landmarks, self.mp_holistic.FACEMESH_TESSELATION, 
                                 self.mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                                 self.mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)) 

        # Draw pose connections
        self.mp_drawing.draw_landmarks(image, results.pose_landmarks, self.mp_holistic.POSE_CONNECTIONS,
                                 self.mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                                 self.mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)) 

        # Draw left hand connections
        self.mp_drawing.draw_landmarks(image, results.left_hand_landmarks, self.mp_holistic.HAND_CONNECTIONS, 
                                 self.mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                                 self.mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 

        # Draw right hand connections  
        self.mp_drawing.draw_landmarks(image, results.right_hand_landmarks, self.mp_holistic.HAND_CONNECTIONS, 
                                 self.mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                 self.mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 
        
        
        
    def input_type(self, no_of_type, path = None):
        
        # 0 for live video, 1 for recorded video
        if no_of_type == 0:
            cap = cv2.VideoCapture(no_of_type)

        elif no_of_type == 1:
            cap = cv2.VideoCapture(path)

        return cap
    
    def extract_keypoints(self,results):
        face_landmarks = np.array([[lm.x, lm.y, lm.z] for lm in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)
        pose_landmarks = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
        righ_hand_landmarks = np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
        left_hand_landmarks = np.array([[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
        return np.concatenate([pose_landmarks, face_landmarks, left_hand_landmarks, righ_hand_landmarks])

    def run(self,no_of_type, path = None):
        '''
        no_of_type : 0 for 'live video', 1 for 'recorded video'
        path : add path of recorded video when no_of_type = 1
        '''
    
    # Define the type of the input
        cap = self.input_type(no_of_type , path)

        # Set mediapipe model 
        with self.mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

            while cap.isOpened():

                # Read feed
                ret, frame = cap.read()
                cv2.imshow('Live',frame)

                # Make detections
                image, results = self.mediapipe_detection(frame, holistic)
                
                # np_array_keypoints is a variable which has the array for each frame in real time 
                np_array_keypoints = self.extract_keypoints(results)
            
            
                # Draw landmarks
                self.draw_styled_landmarks(image, results)

                # Show to screen
                cv2.imshow('OpenCV Feed', image)

                # Break gracefullyq
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            
            cap.release()
            cv2.destroyAllWindows()

In [36]:
slt = SLT()

In [37]:
slt.run(0,None)